# Cheap Optical Flow: Is It Good?


## Quickstart

## Credits

Some portions of this notebook adapted from:
 * [Middlebury Flow code by Johannes Oswald](https://github.com/Johswald/flow-code-python/blob/master/readFlowFile.py)
 * [OpenCV Samples](https://github.com/opencv/opencv/blob/master/samples/python/opt_flow.py)

In [ ]:
# parameters

In [ ]:
## Data Model & Utility Code

import attr
import cv2
import imageio
import IPython.display
import os
import PIL.Image
import six

from oarphpy import plotting as op_plt
import numpy as np


@attr.s(slots=True, eq=False, weakref_slot=False)
class OpticalFlowPair(object):
    """A pair of images with an optical flow field"""
    
    dataset = attr.ib(type=str, default='')
    """To which dataset does this pair belong?"""
    
    id1 = attr.ib(type=str, default='')
    """Identifier or URI for the first image"""
    
    id2 = attr.ib(type=str, default='')
    """Identifier or URI for the second image"""
    
    img1 = attr.ib(default=None)
    """URI or numpy array for the first image (source image)"""

    img2 = attr.ib(default=None)
    """URI or numpy array for the second image (target image)"""
    
    flow = attr.ib(default=None)
    """Numpy array representing optical flow from img1 -> img2"""
    
    def to_html(self):
        im1 = load_image(self.img1)
        im2 = load_image(self.img2)
        fviz = draw_flow(im1, self.flow)
        html = """
            <table>
            
            <tr><td style="text-align:left"><b>Dataset:</b> {dataset}</td></tr>
            
            <tr><td style="text-align:left"><b>Source Image:</b> {id1}</td></tr>
            <tr><td><img src="{im1}" width="100%" /></td></tr>

            <tr><td style="text-align:left"><b>Target Image:</b> {id2}</td></tr>
            <tr><td><img src="{im2}" width="100%" /></td></tr>

            <tr><td style="text-align:left"><b>Flow</b></td></tr>
            <tr><td><img src="{fviz}" width="100%" /></td></tr>
            </table>
        """.format(
                dataset=self.dataset,
                id1=self.id1, id2=self.id2,
                im1=img_to_data_uri(im1), im2=img_to_data_uri(im2),
                fviz=img_to_data_uri(fviz))
        return html


## General Utilities
    
def imshow(x):
    IPython.display.display(PIL.Image.fromarray(x))

def show_html(x):
    from IPython.core.display import display, HTML
    display(HTML(x))
    
def load_image(x):
    if isinstance(x, six.string_types):
        return imageio.imread(x)
    else:
        return x

img_to_data_uri = lambda x: op_plt.img_to_data_uri(x, format='png')
# # TODO correct oarphpy img uri to be png or jpeg   data_url = 'data:image/png;base64,{}'.format(parse.quote(data)) 
# def img_to_data_uri(img, format='png', jpeg_quality=75):
#     """Given a numpy array `img`, return a `data:` URI suitable for use in 
#     an HTML image tag."""

#     from io import BytesIO
#     out = BytesIO()

#     import imageio
#     kwargs = dict(format=format)
#     if format == 'jpg':
#         kwargs.update(quality=jpeg_quality)
#     imageio.imwrite(out, img, **kwargs)

#     from base64 import b64encode
#     data = b64encode(out.getvalue()).decode('ascii')

#     from six.moves.urllib import parse
#     data_url = 'data:image/png;base64,{}'.format(parse.quote(data))

#     return data_url


def draw_flow(img, flow, step=8):
    """Based upon OpenCV sample: https://github.com/opencv/opencv/blob/master/samples/python/opt_flow.py"""
    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    vis = img.copy()
    cv2.polylines(vis, lines, 0, (0, 255, 0))
    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
    return vis


## Middlebury Optical Flow



In [ ]:
# Please unzip `other-color-allframes.zip` and `other-gt-flow.zip` to a directory and provide the target below:
MIDD_DATA_ROOT = '/outer_root/host_mnt/Volumes/970-evo-raid0/middlebury-flow/'

# For the Middlebury Flow dataset, we only consider the real scenes
MIDD_SCENES = [
    {
        'input': 'other-data/Dimetrodon/frame10.png',
        'expected_out': 'other-data/Dimetrodon/frame11.png',
        'flow_gt': 'other-gt-flow/Dimetrodon/flow10.flo',
    },
        {
        'input': 'other-data/Hydrangea/frame10.png',
        'expected_out': 'other-data/Hydrangea/frame11.png',
        'flow_gt': 'other-gt-flow/Hydrangea/flow10.flo',
    },
        {
        'input': 'other-data/RubberWhale/frame10.png',
        'expected_out': 'other-data/RubberWhale/frame11.png',
        'flow_gt': 'other-gt-flow/RubberWhale/flow10.flo',
    },
]


def midd_read_flow(file):
    # Based upon: https://github.com/Johswald/flow-code-python/blob/master/readFlowFile.py
    # compute colored image to visualize optical flow file .flo
    # Author: Johannes Oswald, Technical University Munich
    # Contact: johannes.oswald@tum.de
    # Date: 26/04/2017
    # For more information, check http://vision.middlebury.edu/flow/ 

    assert type(file) is str, "file is not str %r" % str(file)
    assert os.path.isfile(file) is True, "file does not exist %r" % str(file)
    assert file[-4:] == '.flo', "file ending is not .flo %r" % file[-4:]
    f = open(file, 'rb')
    flo_number = np.fromfile(f, np.float32, count=1)[0]
    TAG_FLOAT = 202021.25
    assert flo_number == TAG_FLOAT, 'Flow number %r incorrect. Invalid .flo file' % flo_number
    w = np.fromfile(f, np.int32, count=1)
    h = np.fromfile(f, np.int32, count=1)

    #if error try: data = np.fromfile(f, np.float32, count=2*w[0]*h[0])
    data = np.fromfile(f, np.float32, count=int(2*w*h))
    
    # Reshape data into 3D array (columns, rows, bands)
    flow = np.resize(data, (int(h), int(w), 2))	
    f.close()
    
    # We found that there are some invalid (?) (i.e. very large) flows, so we're going
    # to ignore those for this experiment.
    invalid = (flow >= 1666)
    flow[invalid] = 0

    return flow


for i, scene in enumerate(MIDD_SCENES):
    p = OpticalFlowPair(
            dataset="Middlebury Optical Flow",
            id1=scene['input'],
            img1=os.path.join(MIDD_DATA_ROOT, scene['input']),
            id2=scene['expected_out'],
            img2=os.path.join(MIDD_DATA_ROOT, scene['expected_out']),
            flow=midd_read_flow(os.path.join(MIDD_DATA_ROOT, scene['flow_gt'])))
    
    show_html(p.to_html() + "<br/><br/><br/>")
